# Bibliotecas

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [ ]:
!pip install datasets
!pip install emoji
!pip install transformers
!pip install accelerate

!python -m spacy download pt_core_news_sm

2023-07-03 02:03:33.775302: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 40.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import spacy
import re
import numpy as np
import matplotlib.pyplot as plt
import emoji
import os
import torch
import torch.nn.functional as f

from datasets import load_dataset
from unicodedata import normalize
from tqdm import tqdm
from collections import Counter
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import Dataset
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

In [ ]:
import spacy
import re
import numpy as np
import matplotlib.pyplot as plt
import emoji
import os

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from datasets import load_dataset
from unicodedata import normalize
from tqdm import tqdm
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

import torch
from transformers import Trainer, TrainingArguments
import pandas as pd
import csv
import spacy
import emoji
from unicodedata import normalize
from tqdm import tqdm
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import random
from sklearn.model_selection import train_test_split

# Dataset:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

def preprocessar_tweets(tweets):
    nlp = spacy.load('pt_core_news_sm')
    repetion_pattern = re.compile(r'(.)\1\1+')
    new_tweets = []
    with tqdm(total=len(tweets), colour='green', desc='Processando') as pbar:
      for tweet in tweets[1:]:
          tweet[0] = emoji.demojize(tweet[0], language='pt')
          tweet[0] = tweet[0].replace('_', ' ')
          tweet[0] = normalize('NFKD', tweet[0]).encode('ASCII', 'ignore').decode('ASCII')
          tweet[0] = repetion_pattern.sub(r'\1', tweet[0])
          tweet[0] = re.sub(r'@\w+', ' ', tweet[0])
          tweet[0] = re.sub(r'\s\s+', ' ', tweet[0])
          doc = nlp(tweet[0])
          tokens = [t.lemma_.lower() for t in doc if t.pos_ != 'PUNCT' and not t.is_stop and len(t.lemma_) > 1]
          new_tweet = ' '.join(tokens)
          tweet[0]=(tweet[0].strip())
          tweet[1]=int(tweet[1])
          tweet[2]=int(tweet[2])
          tweet[3]=tweet[3].replace(",", ".", 1)
          tweet[3]=int (float(tweet[3]))
          if(tweet[3]==-1):
            tweet[3]=10
          new_tweets.append(tweet)
          pbar.update(1)

    return new_tweets

def ler_csv(arquivo):
    matriz = []

    with open(arquivo, 'r') as csv_file:
        leitor = csv.reader(csv_file)

        for linha in leitor:
            matriz.append(linha)

    return matriz

In [ ]:
matriz = ler_csv('/content/drive/MyDrive/5d/HateBR.csv')
dataset=preprocessar_tweets(matriz)


Processando: 100%|█████████▉| 7000/7001 [00:43<00:00, 162.01it/s]


In [ ]:
X=np.array(dataset)
dataset.pop(0)

['este lixo .', 1, 1, 10]

In [ ]:
X=np.array(dataset)[:,0]
y =  np.array(dataset)[:,[3]]


In [ ]:
print(type(y[5]))

<class 'numpy.ndarray'>


In [ ]:
n_rows=-1
# Separar em conjunto de treinamento e conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Separar em conjunto de treinamento e conjunto de validação
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
tweets_train = (X_train)
tweets_validation = (X_validation)
tweets_test = (X_test)

print(f'\n\nPreprocessed Tweet Train: {X_train[0]}')
print(f'Preprocessed Tweet Validation: {X_validation[0]}')
print(f'Preprocessed Tweet Test: {X_test[0]}')



Preprocessed Tweet Train: Sei nao hein!? Pertinente, porem vejo um desvio de pensamento arquitetado ai!
Preprocessed Tweet Validation: Importante e oportuna homenagem ao mestre de sempre. Paulo Freire vive!
Preprocessed Tweet Test: Esplicar isso para um eleitor do bozo , e mesmo q da remedio a defunto k


In [ ]:
y_train = [[int(num) for num in sublist] for sublist in y_train]
y_validation =  [[int(num) for num in sublist] for sublist in y_validation]
y_test = [[int(num) for num in sublist] for sublist in y_test]

In [ ]:
print(y_train[0])
print(y_validation[0])
print(y_test[0])

[10]
[0]
[0]


In [ ]:
#model_name = 'distilbert'
model_name = 'bert_base'
# model_name = 'bertabaporu'

num_classes = len(np.unique(y))

if model_name == 'distilbert':
  model_path = 'adalbertojunior/distilbert-portuguese-cased'
elif model_name == 'bert_base':
  model_path = 'neuralmind/bert-base-portuguese-cased'
elif model_name == 'bertabaporu':
  model_path = 'pablocosta/bertabaporu-base-uncased'

tokenizer = BertTokenizer.from_pretrained(model_path)

model = BertForSequenceClassification.from_pretrained(model_path, num_labels=num_classes)


print(f'\n\nNúmero de Parâmetros: {model.num_parameters():,}')

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the



Número de Parâmetros: 108,931,595


In [ ]:
y_train = torch.tensor(y_train)
y_val = torch.tensor(y_validation)
y_test = torch.tensor(y_test)

y_train = f.one_hot(y_train, num_classes=num_classes)
y_val = f.one_hot(y_val, num_classes=num_classes)
y_test = f.one_hot(y_test, num_classes=num_classes)

In [ ]:
y_train[1]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [ ]:
y_test[1]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

device

'cpu'

In [ ]:
train_dict = {'text': tweets_train, 'label': y_train}
val_dict = {'text': tweets_validation, 'label': y_val}
test_dict = {'text': tweets_test, 'label': y_test}

train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)
test_dataset = Dataset.from_dict(test_dict)

In [ ]:
def tokenize_tweet(example, tokenizer, max_len):
  return tokenizer(example['text'], padding='max_length', max_length=max_len, truncation=True)

In [ ]:
max_len = 150
batch_size = 32

encoded_train_dataset = train_dataset.map(lambda t: tokenize_tweet(t, tokenizer, max_len), \
                                  batched=True, batch_size=batch_size)

encoded_val_dataset = val_dataset.map(lambda t: tokenize_tweet(t, tokenizer, max_len), \
                                      batched=True, batch_size=batch_size)

encoded_test_dataset = test_dataset.map(lambda t: tokenize_tweet(t, tokenizer, max_len),
                                        batched=True, batch_size=batch_size)

Map:   0%|          | 0/4199 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [ ]:
encoded_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4199
})

In [ ]:
for item in encoded_val_dataset:
    print(item['label'])


[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
encoded_test_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1400
})

In [ ]:
def computar_medidas_eval(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  labels = np.argmax(labels, axis=-1)
  acc = accuracy_score(labels, predictions)
  return {
      'acc': acc
  }

In [ ]:
batch_size

32

In [ ]:
num_epochs = 10
training_args = TrainingArguments(output_dir='training/', num_train_epochs=num_epochs,
                                  weight_decay=0.01,
                                  learning_rate=5e-5,
                                  optim='adamw_torch',
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  logging_steps=100, eval_steps=100,
                                  evaluation_strategy='epoch',
                                  save_strategy='epoch', load_best_model_at_end=True,
                                  metric_for_best_model='acc', greater_is_better=True)

trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=encoded_train_dataset,
                  eval_dataset=encoded_val_dataset,
                  compute_metrics=computar_medidas_eval,
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=3)])
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 19>:19                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2759 in training_step            │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                                                           │
│   2762 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2784 in compute_loss             │
│                                                                                                  │
│   2781 │   │   │   labels = inputs.pop("labels")                                                 │
│   2782 │   │   else:                                                                             │
│   2783 │   │   │   labels = None                                                                 │
│ ❱ 2784 │   │   outputs = model(**inputs)                                                         │
│   2785 │   │   # Save past state if it exists              

In [ ]:
encoded_val_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1400
})

In [ ]:
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2759 in training_step            │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                                                           │
│   2762 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2784 in compute_loss             │
│                                                                                                  │
│   2781 │   │   │   labels = inputs.pop("labels")                                                 │
│   2782 │   │   else:                                                                             │
│   2783 │   │   │   labels = None                                                                 │
│ ❱ 2784 │   │   outputs = model(**inputs)                                                         │
│   2785 │   │   # Save past state if it exists              

In [ ]:
y_pred, _, _ = trainer.predict(encoded_test_dataset)

y_pred = np.argmax(y_pred, axis=-1)

y_pred = [y for y in y_pred]

In [ ]:
report = classification_report(y_test, y_pred, zero_division=0)

print(report)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

plt.show()